In [1]:
using Images
using PyPlot
using Statistics
using LsqFit
using DelimitedFiles

#setting the correct GUI backend for proper figure display
include("utils.jl")
set_gui();

In [48]:
#provide a path to the kymograph
img_path = "C:/Users/vavol/Documents/work/art/our papers/NS methods paper/NS_examples/dynein/170705/"
img_name = "170705 ch1 exp6 kymo1.tif"
loadpath = string(img_path,img_name);

timestep = 1.34;  #dt between kymograph lines in seconds
mag = 0.107;   #pixel size in micrometers

#which channel of the kymograph contains moving particles to trace?
channel = 2;

#load the full tiff and display the channel we will be analyzing
img = Float32.(load(loadpath));
img_a = Float32.(img);
kymograph = copy(img_a[1:160,:,channel]);
imshow(kymograph);

In [5]:
#manually select a portion of the kymograph containing only one moving spot
#click twice on the kymograph: once on top left corner of our future crop, once on bottom right corner
coords = zeros(4);
coords = ginput_routine(2,kymograph)

(1, 158, 20, 43)

In [6]:
#display the cropped kymograph defined by our clicks
crop = kymograph[coords[1]:coords[2],coords[3]:coords[4]];
cropfig = figure();
imshow(crop);

In [56]:
#optional
#fine-tune cropped kymograph
coords = (coords[1], coords[2], coords[3]+3, coords[4])
crop = kymograph[coords[1]:coords[2],coords[3]:coords[4]];
close(cropfig);
cropfig = figure();
imshow(crop);

In [62]:
#record background (BG) intensity
#click twice on the cropped kymograph selecting a rectangle without moving particles
BG_coords = ginput_routine(2,crop)

(1, 11, 20, 24)

In [54]:
#record initial intensity and initial position of the moving particle
#click twice on the cropped kymograph selecting a rectangle containing >=10 initial positions of the moving particle
peak_coords = ginput_routine(2,crop)

(43, 67, 9, 15)

In [63]:
#to get NS length instead of NS end position:
d = mean(crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]);                #mean background intensity
dsd = stdm((crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]), d);         #background SD
t = size(crop,1);
timepix = zeros(t);
c_av = zeros(t);
timesec = zeros(t);
NSposition = zeros(t);
let d = d, dsd = dsd
    for j in 1:t
        ydata = crop[j,:];
        xdata = collect(1:length(ydata));
        s_av = zeros(length(xdata)-2)
        for i in 2:length(xdata)-2
            s_av[i] = mean(ydata[i-1:i+1]);
        end
        (r,c) = findall(pp->pp>d+dsd,s_av);
        c_av[j] = mean(c);
        NSposition[j] = mean(c) * 2 * mag;
        timepix[j] = j;
        timesec[j] = j*timestep;
    end
end
#plot fitted coordinates
figNS = figure(img_name, figsize=(4,8));
subplot(121, title="cropped kymograph", xlabel="position", ylabel="time");
imshow(crop);

subplot(122, title="fitted coordinates", xlabel="position", ylabel="time");
imshow(crop);
ylim(t,0);
xlim(0,length(crop[1,:]));
grid("on");
pl = scatter(c_av, timepix, facecolor="none", edgecolors="red", s = 30);
PyPlot.title(img_name);

#plot position and intensity of the diffusing spot over time
figNS2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(timesec,NSposition, color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(NSposition));
ax = gca()

PyPlot.title(img_name);

In [39]:
#prepare initial values for the spot fitting function  
d = mean(crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]);                #mean background intensity
dsd = stdm((crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]), d);         #background SD
a = maximum(crop[peak_coords[1]:peak_coords[2],peak_coords[3]:peak_coords[4]]) - d; #peak height
b = (peak_coords[3] + peak_coords[4])/2;                                            #peak position
c = - 2;   width_tol = 0.2;                                                         #peak width and its tolerance
t = size(crop,1);                                                                   #number of timepoints (kymograph lines)

#preallocating fitting result arrays for speed
timesec = zeros(t);
timepix = zeros(t);
pixel = zeros(t);
QDposition = zeros(t);
intensity = zeros(t);
peakwidth = zeros(t);

In [40]:
#we fit each line in our cropped kymograph with a gaussian peak of height a and width c at position b
let a = a, b = b, c = c, d = d, dsd = dsd, t = t, width_tol = width_tol
    for i in 1:t
        ydata = crop[i,:];
        xdata = collect(1:length(ydata));
        
        (a, b, c, d) = fit_gaussian_NS(xdata, ydata, c, d, dsd)
        
        pixel[i] = b;
        timesec[i] = i*timestep;
        QDposition[i] = b*mag;
        peakwidth[i] = -c;
        
        if i > 1
            if peakwidth[i] > peakwidth[1]*(1+width_tol)
                peakwidth[i]=peakwidth[1]*(1+width_tol);
            elseif peakwidth[i] < peakwidth[1]*(1-width_tol)
                peakwidth[i]=peakwidth[1]*(1-width_tol);
            end
        end
        
        intensity[i] = a*peakwidth[i];
        
        timepix[i] = i;
        IJulia.clear_output(true);
        println(string("Processing line ", i, " of ", t, "; a = ", round(a), "; b = ", round(b)), "; c = ", round(c))
        
    end
end

Processing line 150 of 150; a = 1152.0; b = 19.0; c = -2.0


In [64]:
#IF USING NS length: plot fitted coordinates
fig35 = figure(img_name, figsize=(4,8));
subplot(121, title="cropped kymograph", xlabel="position", ylabel="time");
imshow(crop);

subplot(122, title="fitted coordinates");
#imshow(crop);
ylim(t,0);
xlim(0,length(crop[1,:]));
#grid("on");
pl = scatter(c_av, timepix, facecolor="none", edgecolors="red", s = 30);
PyPlot.title(img_name);

In [42]:
#plot position and intensity of the diffusing spot over time
fig2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(timesec,QDposition, color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(QDposition));
ax = gca()
p3 = plot(timesec,NSposition, color="orange");

In [43]:
#plot position and intensity of the diffusing spot over time
fig2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(timesec,QDposition, color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(QDposition));
ax = gca()

fig2.subplots_adjust(right=0.85);
ax2 = ax.twinx();
font2 = Dict("color"=>"orange");
ylabel("intensity (a.u.)",fontdict=font2);
p3 = plot(timesec,NSposition, color="orange");
ylim(0, maximum(NSposition));

PyPlot.title(img_name);

In [18]:
#saving NS length intead of position
#check if there's a folder to save data
let savepath = string(img_path, "traced/")
    if isdir(savepath) == false
        mkdir(savepath)
    end 
end

#saving the data: *subcrop.tif and a tab-delimited text file with fitting results
#multiple spots from the same kymograph will be saved with increasing index in the filename
function fcheck()
    filecount = 1;
    savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    while isfile(savename) == true
        filecount = filecount + 1
        savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    end
    return filecount
end

filecount = fcheck();

savenameNS = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt")
open(savenameNS, "w") do io;
    writedlm(io, [timesec NSposition]);
end

subcropname = string(img_path, "traced/", img_name[1:end-4], "-", filecount, "_subcrop.tif");
save(subcropname, colorview(Gray, crop));

In [75]:
#check if there's a folder to save data
let savepath = string(img_path, "traced/")
    if isdir(savepath) == false
        mkdir(savepath)
    end 
end

#saving the data: *subcrop.tif and a tab-delimited text file with fitting results
#multiple spots from the same kymograph will be saved with increasing index in the filename
function fcheck()
    filecount = 1;
    savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    while isfile(savename) == true
        filecount = filecount + 1
        savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    end
    return filecount
end

filecount = fcheck();

savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt")
open(savename, "w") do io;
    writedlm(io, [timesec QDposition intensity]);
end

subcropname = string(img_path, "traced/", img_name[1:end-4], "-", filecount, "_subcrop.tif");
save(subcropname, colorview(Gray, crop));

In [502]:
close("all")

In [ ]:
#optional, checking the saved files
